In [36]:
import config
import tweepy
import pandas as pd
import requests
import datetime

In [397]:
today = datetime.datetime.today()
tomorrow = today + datetime.timedelta(days=1)
dayAfter = today + datetime.timedelta(days=2)
dates = [today.strftime("%d-%m-%Y"),tomorrow.strftime("%d-%m-%Y"),dayAfter.strftime("%d-%m-%Y")]

In [420]:
clean = False


In [399]:
auth = tweepy.OAuthHandler(config.key,config.scret_key)
auth.set_access_token(config.access_token,config.access_token_secret)
api = tweepy.API(auth)

In [424]:
df = pd.read_csv("UserData.csv")
mentions = api.mentions_timeline()
for mention in mentions:
    splitt = mention.text.split(" ")
    username = mention.user.screen_name
    tweetID = mention.id
    if df["TweetID"].any()==tweetID :
        print("Tweet Handled")
        continue
    if clean:
        try:
            api.destroy_status(tweetID)
            print("Clean: Deleted")
        except:
            print("Clean: Skipped")
            continue
    else:
        try:
            pin = splitt[2]
            age = int(splitt[3].split("+")[0])
            reply = "@" + username + " You have been successfuly subscribed. I will be reminding you when vaccine slot is available at your pincode " + pin
            if df["Username"].any()==username :
                reply = "@" + username + " " + "You are already subscribed"
            else:
                df = df.append({ "Username": username,"TweetID": tweetID, "Pin": pin, "Age": age},ignore_index=True)
            api.update_status(reply,tweetID)
            print(pin,age,username,tweetID)
        except:
            print("Wrong tweet")
            reply = "@" + username + " " + 'There seems to be an error. Please make sure your tweet is in the following format ' + "errorid:" + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            file = open("ss.jpeg" , "rb")
            r1 = api.media_upload(filename="ss.jpeg",file=file)
            api.update_status(reply,tweetID,media_ids=[r1.media_id_string])
            continue

df.to_csv("UserData.csv",index=False)

110008 45 get_dash 1389572294318510084


In [429]:
baseURL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin?"
for date in dates:
    for pin in df["Pin"].unique():

        completeURL = baseURL+"date="+date + "&pincode=" + str(pin)
        try:
            sessions = requests.get(completeURL).json()["sessions"]
        except:
            print("Failed to get new sessions")

        if sessions:
            vaccines = {"COVISHIELD" : False, "COVAXIN" : False}
            minAge = {18:False,45:False}
            availableCapacity = sys.maxsize

            for session in sessions:
                availableCapacity = min(availableCapacity,int(session["available_capacity"]))
                vaccines[session["vaccine"]] = True
                minAge[session["min_age_limit"]] = True
            if minAge[18]:
                tweetAt(availableCapacity,vaccines,pin,date,18)
            if minAge[45]:
                tweetAt(availableCapacity,vaccines,pin,date,45)

        else:
            continue

@get_dash Atleast 126 slot(s) for COVISHIELD is available for age 45+ at pin 110008 for 04-05-2021
@get_dash Atleast 108 slot(s) for COVISHIELD is available for age 45+ at pin 110008 for 05-05-2021
@get_dash Atleast 21 slot(s) for COVISHIELD and COVAXIN is available for age 45+ at pin 110008 for 06-05-2021


In [428]:
def tweetAt(availableCapacity,vaccines,pin,date,age):

    reply = "Atleast " + str(availableCapacity) + " slot(s) for " + " and ".join([v for v,b in vaccines.items() if b == True]) + " is available for age " + str(age) + "+ at pin " + str(pin) + " for " + date
    tweetIDs = df.loc[(df["Age"]==age) & (df["Pin"]==pin),["TweetID"]]

    for tweetID in tweetIDs.TweetID:
        username = df.loc[df["TweetID"]==tweetID,["Username"]].Username.item()
        tweet = "@" + username + " " + reply
        print(tweet)
        api.update_status(tweet,tweetID)





In [375]:
sessions[0]["min_age_limit"] = 18

In [362]:
df

,Username,TweetID,Pin,Age
0,get_dash,13893898628267134967,110011,18
1,get_dash,138938886282671634977,110011,18
2,get_dash,13893886282671934977,110012,18
3,get_dash,138938866282671634977,110012,45
4,hello,13893886282676134977,110012,45


In [121]:
print(df["Age"].any()==45)

False


In [255]:
age45 =

In [288]:
len(sessions)

3

In [176]:

age45.TweetID

3    1389388628267134977
4    1389388628267134977
Name: TweetID, dtype: int64

In [181]:
age45.Username.get()

TypeError: get() missing 1 required positional argument: 'key'

In [282]:
session["center_id"]

1281

In [202]:
age45[["Username"]["Age"]]

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<ipython-input-202-eb6e3c6c9b12>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  age45[["Username"]["Age"]]


TypeError: list indices must be integers or slices, not str

In [188]:
age45["TweetID"]

3    1389388628267134977
4    1389388628267134977
Name: TweetID, dtype: int64

In [232]:
age45.TweetID

3    1389388628267134977
4    1389388628267134977
Name: TweetID, dtype: int64

,TweetID
3,138938866282671634977
4,13893886282676134977


In [272]:
df["Pin"]==110018 & df["Age"]==45

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [270]:
df["Age"]==45

0    False
1    False
2    False
3     True
4     True
Name: Age, dtype: bool

In [352]:
vaccines = ""
vaccines = {"COVISHIELD" : False, "COVAXIN" : False}
covaxinAvailable = False

availableCapacity = sys.maxsize
for session in sessions:
    availableCapacity = min(availableCapacity,int(session["available_capacity"]))
    vaccines[session["vaccine"]] = True

reply = "Atleast " + str(availableCapacity) + " slot(s) for " + " and ".join([v for v,b in vaccines.items() if b == True]) + " at your locality " + str(pin) + " for " + date
print(availableCapacity,vaccines)
print(reply)

1 {'COVISHIELD': True, 'COVAXIN': True}
Atleast 1 slot(s) for COVISHIELD and COVAXIN at your locality 110012 for 4-05-2021


In [325]:
session["vaccine"] = "COVAXIN"

In [342]:
vax = [v for v,b in vaccines.items() if b == True]

In [338]:
len([v for v,b in vaccines.items() if b == True]) > 0


True

In [344]:
"hello" + vax

TypeError: can only concatenate str (not "list") to str

In [348]:
" and ".join(vax)

'Hello'

In [347]:
vax = ["Hello"]

'20210504185125'

In [430]:
api.update_status("Hello World")

Status(_api=<tweepy.api.API object at 0x12341d1c0>, _json={'created_at': 'Tue May 04 13:36:30 +0000 2021', 'id': 1389574658224128004, 'id_str': '1389574658224128004', 'text': 'Hello World', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'source': '<a href="https://help.twitter.com/en/using-twitter/how-to-tweet#source-labels" rel="nofollow">getThisSlot</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1389340423332659206, 'id_str': '1389340423332659206', 'name': 'CowinHelper', 'screen_name': 'getmyslot', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Mon May 03 22:05:54 +0000 2021', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'ver